# Clasificación de dígitos en Keras

Importar datos

In [1]:
#import mnist
import numpy as np
import tensorflow as tf

from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

#train_images = mnist.train_images()

#train_labels = mnist.train_labels()

#test_images = mnist.test_images()
#test_labels = mnist.test_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## Preprocesar los datos

In [2]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

print(train_images.shape)
print(test_images.shape)

(60000, 784)
(10000, 784)


## Construir el modelo


`model = Sequential([  #layers...])`

El constructor de Sequential recibe como parámetro un arreglo de [Layers](https://keras.io/layers/about-keras-layers/) de Keras

Para el caso de una red neuronal feed forward solo se requiere de la capa [Dense](https://keras.io/layers/core/#dense), que representa una capa completamente conectada.

En este ejemplo añadiremos tres capas al modelo, las primeras dos con 64 nodos cada unoa con la función de activación [relu](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7) y la última con 10 nodos y función de activación [softmax](https://victorzhou.com/blog/softmax/).

Keras requiere conocer la forma de los datos de entrada por medio del parámetro `input_shape` especificado en la primera capa.

In [3]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

2022-03-22 03:07:15.029295: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-22 03:07:15.041485: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 03:07:15.147006: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Compilar el modelo
Es necesario configurar el proceso de entrenamiento considerando tres factores:
- El *optimizador*.
- La *función de pérdida (loss function)*.
- Las *métricas* de optimización.

In [4]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

## Entrenar el modelo
Para entrenar el modelo se invoca la función `fit` con una serie de [parámetros](https://keras.io/models/sequential/#fit), entre los que destacan:
- *Datos de entrenamiento*, incluyendo datos y etiquetas.
- *Número de épocas (epochs)*, que representa el número de iteraciones sobre el dataset completo.
- *El tamaño del batch* número de muestras por actualización de gradiente.

Para este caso Keras espera los targets de entrenamiento como un vector de 10 dimensiones, dado que hay 10 nodos en la capa de salida Softmax, sin embargo train_labels provee de un solo entero que representa la clase de cada imagen. Es por ello que se hace uso del método [to_categorical](https://keras.io/utils/#to_categorical) que convierte al arreglo de enteros en un arreglo de vectores one-hot., por ejemplo 2 estaría representado como `[0,0,1,0,0,0,0,0,0,0,0]` (indexado en cero).

In [5]:
model.fit(
  train_images, # training data
  to_categorical(train_labels), # training targets
  epochs=10,
  batch_size=32,
)

2022-03-22 03:07:22.318827: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-22 03:07:22.438109: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2593905000 Hz


Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.5936 - accuracy: 0.8224
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2009 - accuracy: 0.9404
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1446 - accuracy: 0.9548
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1230 - accuracy: 0.9618
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1030 - accuracy: 0.9687
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0905 - accuracy: 0.9709
Epoch 7/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0824 - accuracy: 0.9740
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0757 - accuracy: 0.9759
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0695 - accuracy: 0.9775
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.062

### Probar el modelo
Keras provee el método [evaluate](https://keras.io/models/sequential/#evaluate) que regresa un arreglo con la pérdida en prueba (test loss) seguida de las métricas especificadas.

In [6]:
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

313/313 [==============================] - 1s 2ms/step - loss: 0.1153 - accuracy: 0.9662


[0.11529228836297989, 0.9661999940872192]

In [7]:
np.shape(test_images)

(10000, 784)

### Usar el modelo
El modelo puede guardarse y recuperarse de disco por medio de las funciones `save_weights` y `load_weights`

La función `predict` de un modelo permite predecir con datos de prueba o instancias nuevas.

In [8]:

# Predict on the first 5 test images.
predictions = model.predict(test_images[:100])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 8 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3 7 4 3 4 3 0 7 0 2 9
 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 9 3 1 4 1 7 6 9]
[7 2 1 0 4]
